In [2]:
%matplotlib inline
import sympy
import numpy as np
import astropy.units as u
import uncertainties as unc
import uncertainties.unumpy as unp
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar
from fact.analysis import li_ma_significance
sympy.init_printing()

## Sensitivity Calculation
The sensitvity of an IACT is defined as the minimum flux needed to observe a (point) source with a significance of least $\sigma_{target}$ within a fixed amount of time $t_{ref}$.
After analysing the high-level IACT data you're left with the number of events from the source region from the sky and the number of events from the background region in the sky. Namely $N_{on}$ and  $N_{off}$. Usually, if not always, the source and background regions have different sizes. The ration between these is called $\alpha$.

The good old Li&Ma formula calculates the significance of an observation based on the numbers above. To calculate the sensitivity we scale the number of 'gamma' events by the relative flux. 

All other counts are scaled taking into account the reference time (i.e. 50 hours) 
and the actual observation time (which of course only really defined for real observations)

The target significance is pre defined and fixed (usually set to 5 sigma). So we create an equality between the target sigma and the scaled Li&Ma formula and solve for the flux.

In [3]:
#target_significance = sympy.Symbol('\sigma_\mathrm{target}')
#eq = sympy.Equality(scaled_li_ma, target_significance)

Calling `sympy.solve(eq, relative_flux)`, to solve for the relative flux, yields no solution unfortunately. So we use a numerical minimizer to solve the problem.

Below you will find a function implementing the Li&Ma formula and a function minimizing the function versus the target significance of 5 sigma. 

In [4]:
@u.quantity_input(t_obs=u.hour, t_ref=u.hour)
def relative_sensitivity(
        n_on,
        n_off,
        alpha,
        target_significance=5,
        ):
    '''
    Calculate the relative sensitivity defined as the flux
    relative to the reference source that is detectable with
    significance in t_ref.

    Parameters
    ----------
    n_on: int or array-like
        Number of signal-like events for the on observations
    n_off: int or array-like
        Number of signal-like events for the off observations
    alpha: float
        Scaling factor between on and off observations.
        1 / number of off regions for wobble observations.
    target_significance: float
        Significance necessary for a detection
        
    Returns
    ----------
    The relative flux neccessary to detect the source with the given target significance.
    '''
    scale = []
    for on, off in zip(n_on, n_off):
        def f(relative_flux):
            s = li_ma_significance((on - off) * relative_flux + off, off, alpha=alpha)
            return s

        s = minimize_scalar(lambda x: (target_significance - f(x))**2)

        if not s.success:
            print('This went horribly wrong.')
        
        scale.append(s.x)
    return scale

In [5]:
# unfortunately the results do not make sense at all.
n_on =  [1, 2, 4, 1, 200, 400]
n_off = [2, 2, 2, 0, 10, 10]

s = relative_sensitivity(n_on, n_off, alpha=1,target_significance=5)
print('relative sensitivity is: {}'.format(s))

relative sensitivity is: [2.6180339603380443, 2.6180339603380443, 12.239475552474291, 2.6180339603380443, -1.6180339754913822, -1.6180339754913822]
